In [ ]:
import pandas as pd
from typing import List
import numpy as np
import json

# путь к файлу
file_path = "../data/movies_clean.jsonl"
data = pd.read_json(file_path, lines=True)

# создаем DataFrame
df = pd.DataFrame(data)

df["meta.tmdb_id"] = "https://www.themoviedb.org/movie/" + df["meta.tmdb_id"].astype(str)
df['meta.year'] = df['meta.year'].fillna(0)  # если есть NaN
df['meta.year'] = df['meta.year'].astype(int)

# объединяем поля в единый текст
def make_doc(row):
    parts = []
    if row.get('title_ru'): parts.append(f"{row['title_ru']}")
    if row.get('meta.year'): parts.append(f"Год выпуска: {row['meta.year']}")
    if row.get('overview_ru'): parts.append(row['overview_ru'])
    if row.get('genres'): parts.append(f"Режиссер: {row['genres']}")
    if row.get('directors'): parts.append(f"Актерский состав: {row['directors']}")
    if row.get('actors_main'): parts.append(f"Жанры: {row['actors_main']}")
    if row.get('keywords'): parts.append(f"keywords: {row['keywords']}")
    return "\n".join([p for p in parts if p])

df['combined_text'] = df.apply(make_doc, axis=1)



# Поменяйте порядок колонок meta, которые хотите сохранить
meta_cols = ['meta.tmdb_id','meta.poster_url','title_ru','overview_ru','meta.year','directors','actors_main','genres']
meta = df[meta_cols].to_dict(orient='records')
docs = df['combined_text'].tolist()

In [ ]:
!pip install sentence-transformers langchain faiss-cpu symspellpy tqdm pandas streamlit pyarrow joblib
!pip install transformers accelerate
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 134.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 116.9 MB/s eta 0:00:00


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np

# создаём embeddings модель
embeddings_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={"device": "cuda"},       # используем CPU
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64}  # меньше batch для CPU
)

# получение эмбеддингов батчами
embeddings = embeddings_model.embed_documents(docs)  # List[List[float]]
embeddings = np.array(embeddings, dtype=np.float32)

In [10]:
np.save("movie_embeds.npy", embeddings)
import joblib
joblib.dump(meta, "movie_meta.pkl")

['movie_meta.pkl']

In [ ]:
import faiss
dim = embeddings.shape[1]

# выбираем индекс: простой flat
index = faiss.IndexFlatIP(dim)  # используем косинус через нормализованные векторы -> inner product

index.add(embeddings)        # добавляем все векторы

faiss.write_index(index, "faiss_index.bin")

In [ ]:
from symspellpy import SymSpell, Verbosity
import pkg_resources

# инициализация
max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)

# подготовим словарь: названия фильмов, жанры
vocab = set()
for s in df['title_ru'].str.split().explode().dropna().unique():
    vocab.add(str(s).lower())
for s in df['genres'].str.split(',').explode().dropna().unique():
    vocab.add(str(s).lower())

# загрузим словарь в symspell
for w in vocab:
    sym_spell.create_dictionary_entry(w, 1)

# функция исправления запроса
def symspell_correct_query(query: str, max_suggestions=3):
    words = query.split()
    corrected_words = []
    for w in words:
        suggestions = sym_spell.lookup(w.lower(), Verbosity.TOP, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)
        else:
            corrected_words.append(w)
    return " ".join(corrected_words)

In [ ]:
from sentence_transformers import CrossEncoder

# cross-encoder для ранжирования
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", device='cpu')  # или cpu

# загрузка индекса и метаданных, если в другой сессии:
# index = faiss.read_index("faiss_index.bin")
# embeddings = np.load("movie_embeds.npy")
# meta = joblib.load("movie_meta.pkl")

def search(query: str, top_k_faiss=50, top_k_return=10, apply_symspell=True):
    q0 = query
    if apply_symspell:
        q = symspell_correct_query(query)
    else:
        q = query

    # embed query
    try:
        q_emb = embeddings_model.embed_query(q)  # если HuggingFaceEmbeddings
    except:
        q_emb = model.encode(q, convert_to_numpy=True, normalize_embeddings=True)

    q_emb = np.array(q_emb, dtype=np.float32).reshape(1, -1)

    # Faiss search (inner product)
    D, I = index.search(q_emb, top_k_faiss)
    candidate_idxs = I[0].tolist()

    # prepare pairs for reranker: (query, doc_text) or (query + title, snippet)
    candidates = []
    for idx in candidate_idxs:
        if idx < 0: continue
        text = docs[idx]
        candidates.append((text, idx))

    # Cross encoder expects list of [query, doc_text]
    cross_inputs = [[q0, docs[idx]] for _, idx in candidates]
    rerank_scores = cross_encoder.predict(cross_inputs)  # higher = more relevant

    # combine and sort
    ranked = sorted(zip([idx for _, idx in candidates], rerank_scores), key=lambda x: x[1], reverse=True)
    top_ranked = ranked[:top_k_return]

    results = []
    for idx, score in top_ranked:
        m = meta[idx]
        results.append({
            'score': float(score),
            'idx': int(idx),
            'title_ru': m.get('title_ru'),
            'meta.year': m.get('meta.year'),
            'meta.tmdb_id': m.get('meta.tmdb_id'),
            'meta.poster_url': m.get('meta.poster_url'),
            'directors': m.get('directors'),
            'actors_main': m.get('actors_main'),
            'genres': m.get('genres'),
            'snippet': docs[idx][:400]  # первые 400 символов
        })
    return results

In [ ]:
!pip install langchain_groq

In [ ]:
SYSTEM_PROMPT = """Ты — киновед и рекомендатель. Тебе даётся список фильмов с краткими описаниями.
Задача: по запросу пользователя дать развернутый ответ: 1) почему эти фильмы релевантны, 2) дополнительные рекомендации, 3) связи между фильмами (жанры, темы), 4) краткий список похожих фильмов, которых может и не быть в выдаче.
Отвечай на русском, структурированно (заголовки)."""

def build_rag_context(results, user_query, max_chars=1800):
    parts = []
    for r in results:
        s = f"Title: {r['movie_title']} ({r.get('year','')})\nGenres: {r.get('genres','')}\nSnippet: {r.get('snippet')}\nURL: {r.get('page_url')}\n"
        parts.append(s)
    context = "\n\n".join(parts)
    # усекаем, если слишком длинно
    if len(context) > max_chars:
        context = context[:max_chars]
    prompt = f"{SYSTEM_PROMPT}\n\nUser query: {user_query}\n\nContext:\n{context}\n\nОтвет:"
    return prompt

# пример использования с ChatGroq
import os, getpass
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

os.environ["GROQ_API_KEY"] = getpass.getpass("")

llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0, max_tokens=4096)

def rag_answer(user_query, results):
    prompt = build_rag_context(results, user_query)
    messages = [SystemMessage(content=SYSTEM_PROMPT), HumanMessage(content=f"Query: {user_query}\n\nContext:\n{build_rag_context(results, user_query)}")]
    resp = llm(messages)
    return resp.content 

In [ ]:
queries = [
    "комедия",
    "космос",
    "романтическая комедия в большом городе",
    "фильм про мальчика с волшебной палочкой",
    "история о кольце власти",
    "Гарри Поттер",
    "Marvel фильмы",
    "хорор фелм",   
    "комидея"       
]

for q in queries:
    print("=== QUERY:", q)
    res = search(q, top_k_faiss=70, top_k_return=5, apply_symspell=True)
    for r in res:
        print(r['title_ru'], "-", r['meta.year'], "score:", r['score'])
    print()

=== QUERY: комедия
Майк Бирбиглия: Слава богу, есть шутки - 2017.0 score: 6.763744831085205
Поймай толстуху, если сможешь - 2013.0 score: 6.5741286277771
Порнократия - 2004.0 score: 6.4279093742370605
Почему ты улыбаешься? - 2024.0 score: 6.30178689956665
Кошелёк или жизнь - 2007.0 score: 6.225667476654053

=== QUERY: космос
Астронавт - 2025.0 score: 7.3118896484375
Космический Джем: Новое Поколение - 2021.0 score: 7.199644565582275
Космические чистильщики - 2021.0 score: 7.039042949676514
В космосе с Маркиплиером: Часть 2 - 2022.0 score: 6.852969169616699
Что осталось после нас: оглядываясь на "Звёздный путь: Дальний космос 9" (2018) - 2018.0 score: 6.668680667877197

=== QUERY: романтическая комедия в большом городе
Рождественский роман с ковбоем - 2023.0 score: 9.022026062011719
Римские свидания - 2016.0 score: 8.857696533203125
Стильная штучка - 2002.0 score: 8.702308654785156
Зачарованная - 2007.0 score: 8.565037727355957
Больше, чем друг - 2010.0 score: 8.562870979309082

=== QUE